In [ ]:
from tensorflow import keras
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import tensorflow.keras.backend as K
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
dataset = pd.read_csv("../Datasets/housing.csv")

dataset = dataset.dropna(axis=0)
dataset = dataset.drop('ocean_proximity',axis=1)
dataset = shuffle(dataset)
x_test = dataset.iloc[:2000,:]
x_train = dataset.iloc[2000:,:]
y_train = x_train.pop("median_house_value")
y_test = x_test.pop("median_house_value")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
y_train = np.expand_dims(y_train,axis=1)
y_test = np.expand_dims(y_test,axis=1)
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)


In [ ]:
model = models.Sequential()

model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(20 ,activation='relu'))
model.add(layers.Dense(15 ,activation='relu'))
model.add(layers.Dense(1,activation=keras.activations.softsign))

In [ ]:
from tensorflow.keras import backend as K
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [ ]:
lrs = []
K1 = 0.
batch_size=16
epochs = 1000

In [ ]:
def lr_schedule(epoch):
    global K1

    Kz = 0.
    
    for i in (range((len(x_train) - 1) // batch_size + 1)):
      start_i = i * batch_size
      end_i = start_i + batch_size
      xb = x_train[start_i:end_i]
    
      tmp = np.array(func([xb]))
      activ = np.linalg.norm(tmp)
      
      if activ > Kz:
          Kz = activ
    print("The value of K_z is ",Kz)
    K1 = (1/batch_size)*(Kz)*max(q,1-q)
    lr = 1 / K1
    lrs.append(lr)
    print('Epoch', epoch, 'LR =', lr)
    return lr


In [ ]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [ ]:
q = 0.95
model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer=keras.optimizers.SGD())

In [ ]:
history_const=model.fit(x_train, y_train, epochs = epochs, verbose=0, batch_size = 16,validation_data=(x_test,y_test))

In [ ]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_data=(x_test, y_test), callbacks=[lr_scheduler])

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss.png')
plt.show()


NameError: name 'history' is not defined

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss.png')
plt.show()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
loss_const = history_const.history['loss']
loss_lalr = history.history['loss']

epochs = range(len(loss))

plt.plot(epochs, loss_const, 'r', label='Loss Constant')
plt.plot(epochs, loss_lalr, 'b', label='Loss LALR')
plt.title('Training and validation loss')
plt.ylim(0.15,0.25)
plt.legend()
plt.savefig('quantile_5.png')
plt.show()


Batch size =128

In [ ]:
q = 0.05
model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer=keras.optimizers.SGD())
history_16=model.fit(x_train, y_train, epochs = 1000, batch_size = 16,validation_data=(x_test,y_test),callbacks=[lr_scheduler])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
loss = history_16.history['loss']
val_loss = history_16.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss.png')
plt.show()


In [ ]:
q = 0.05
model.compile(loss=lambda y,f: tilted_loss(q,y,f), optimizer=keras.optimizers.SGD())
history_const_16=model.fit(x_train, y_train, epochs = 1000, batch_size = 16,validation_data=(x_test,y_test))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
loss_const = history_const_16.history['loss']
loss_lalr = history_16.history['loss']

epochs = range(len(loss))

plt.plot(epochs, loss_const, 'r', label='Loss Constant')
plt.plot(epochs, loss_lalr, 'b', label='Loss LALR')
plt.title('Training and validation loss')
plt.ylim(0.04,0.1)
plt.legend()
plt.savefig('quantile_5.png')
plt.show()
